# COGS 108 FINAL PROJECT

This notebook will explore the salary of SF workers in the government sector, and help us understand how these salaries have grown relative to each other, as well as relative to COA, inflation, and general economic growth.

This dataset will come from the years of 2011 - 2014, and will reflect the years post economic downturn from the 2008 recession.

In [2]:
import pandas as pd
import seaborn
import matplotlib as mpl
import numpy as np
import re

We'll need to import the data collected from these Government-related jobs from 2011 - 2014 from the Dataset from Kaggle.

In [3]:
salary_df = pd.read_csv("Salaries.csv", dtype={'JobTitle': 'object', "BasePay": 'object', "Overtime": 'object', 'Otherpay': 'object', 'Agency': 'object'})

C:\Users\UNKNWN\Anaconda3\envs\37cogs108\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (4,5,6,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# print(salary_df.loc[10])
# Explore CPI, Median Home Prices, Utility Prices

We'll also want to clean up this data in regards to safe harbor; this will require us to remove the name of the employee, as well as the agency of each person; conviently, all agencies are in San Francisco, as it is the area we are taking a look at.

In [5]:
salary_df = salary_df.drop(columns=['EmployeeName'])

In [6]:
print(salary_df.loc[10])

Id                                                                11
JobTitle            ASSISTANT CHIEF OF DEPARTMENT, (FIRE DEPARTMENT)
BasePay                                                    194999.39
OvertimePay                                                  71344.9
OtherPay                                                     33149.9
Benefits                                                         NaN
TotalPay                                                      299494
TotalPayBenefits                                              299494
Year                                                            2011
Notes                                                            NaN
Agency                                                 San Francisco
Status                                                           NaN
Name: 10, dtype: object


In [7]:
print(salary_df['JobTitle'].value_counts())
salary_df['JobTitle'].unique()

Transit Operator                                      7036
Special Nurse                                         4389
Registered Nurse                                      3736
Public Svc Aide-Public Works                          2518
Police Officer 3                                      2421
Custodian                                             2418
TRANSIT OPERATOR                                      2388
Firefighter                                           2359
Recreation Leader                                     1971
Patient Care Assistant                                1945
Deputy Sheriff                                        1933
Police Officer                                        1476
SPECIAL NURSE                                         1402
Public Service Trainee                                1328
REGISTERED NURSE                                      1219
Police Officer 2                                      1141
Attorney (Civil/Criminal)                             11

array(['GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY',
       'CAPTAIN III (POLICE DEPARTMENT)',
       'WIRE ROPE CABLE MAINTENANCE MECHANIC', ..., 'Conversion',
       'Cashier 3', 'Not provided'], dtype=object)

Lets try to even out the data by stripping out anything within parenthesis to clear up and duplicates

In [8]:
salary_df['JobTitle'] = salary_df['JobTitle'].str.upper()
print(salary_df['JobTitle'])
salary_df['JobTitle'].value_counts()

0           GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY
1                          CAPTAIN III (POLICE DEPARTMENT)
2                          CAPTAIN III (POLICE DEPARTMENT)
3                     WIRE ROPE CABLE MAINTENANCE MECHANIC
4             DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)
5                                ASSISTANT DEPUTY CHIEF II
6                       BATTALION CHIEF, (FIRE DEPARTMENT)
7                           DEPUTY DIRECTOR OF INVESTMENTS
8                       BATTALION CHIEF, (FIRE DEPARTMENT)
9                   CHIEF OF DEPARTMENT, (FIRE DEPARTMENT)
10        ASSISTANT CHIEF OF DEPARTMENT, (FIRE DEPARTMENT)
11                         CAPTAIN III (POLICE DEPARTMENT)
12                             EXECUTIVE CONTRACT EMPLOYEE
13                                       DEPARTMENT HEAD V
14                      BATTALION CHIEF, (FIRE DEPARTMENT)
15                      COMMANDER III, (POLICE DEPARTMENT)
16                                       DEPARTMENT HEAD

TRANSIT OPERATOR                                    9424
SPECIAL NURSE                                       5791
REGISTERED NURSE                                    4955
CUSTODIAN                                           3214
FIREFIGHTER                                         3153
RECREATION LEADER                                   2663
DEPUTY SHERIFF                                      2618
PUBLIC SVC AIDE-PUBLIC WORKS                        2518
POLICE OFFICER 3                                    2421
PATIENT CARE ASSISTANT                              1945
PUBLIC SERVICE TRAINEE                              1656
ATTORNEY (CIVIL/CRIMINAL)                           1503
POLICE OFFICER                                      1476
PORTER                                              1465
GENERAL LABORER                                     1410
GARDENER                                            1187
POLICE OFFICER 2                                    1141
PARKING CONTROL OFFICER        

In [9]:
#Seperate Dataframes into yearly parts (2011, 2012, 2013, 2014)
year2011 = 0
year2012 = 0
year2013 = 0
year2014 = 0


for i in range(0, salary_df['Year'].size):
    if salary_df['Year'].iloc[i] == 2011:
        year2011 += 1
        year2012 += 1
        year2013 += 1
    elif salary_df['Year'].iloc[i] == 2012:
        year2012 += 1
        year2013 += 1
    elif salary_df['Year'].iloc[i] == 2013:
        year2013 += 1
        
salary_2011 = salary_df[:year2011]
salary_2012 = salary_df[year2011:year2012]
salary_2013 = salary_df[year2012:year2013]
salary_2014 = salary_df[year2013:]

In [10]:
print(salary_2011[salary_2011['JobTitle'] == "CLERK"].index)

Int64Index([ 7949, 20821, 21304, 22666, 22692, 23183, 23506, 23680, 23867,
            24021,
            ...
            34934, 35016, 35041, 35073, 35373, 35603, 35899, 35939, 36001,
            36149],
           dtype='int64', length=219)


Lets drop any counts where the job count is below 30, for clarity sake.

In [11]:
# print(salary_2011)
print(salary_2011["JobTitle"].value_counts())
# drop2011 = []
# drop2012 = []
# drop2013 = []
# drop2014 = []

for key, value in salary_2011["JobTitle"].value_counts().to_dict().items():
    if value <= 30:      
        salary_2011.drop(salary_2011[salary_2011["JobTitle"] == key].index, inplace=True)
    
print(salary_2011["JobTitle"].value_counts())

for key, value in salary_2012["JobTitle"].value_counts().to_dict().items():
    if value <= 30:
#         drop2012.append(key)
        salary_2012.drop(salary_2012[salary_2012["JobTitle"] == key].index, inplace=True)
        
for key, value in salary_2013["JobTitle"].value_counts().to_dict().items():
    if value <= 30:
#         drop2013.append(key)
        salary_2013.drop(salary_2013[salary_2013["JobTitle"] == key].index, inplace=True)

for key, value in salary_2014["JobTitle"].value_counts().to_dict().items():
    if value <= 30:
#         drop2014.append(key)
        salary_2014.drop(salary_2014[salary_2014["JobTitle"] == key].index, inplace=True)
        
# print(salary_2011["JobTitle"].value_counts())
#counter = 0
# # regex = re.compile(r'\([^)]*\)')
# for job in salary_df["JobTitle"]:
#     if regex.search(job):
#         salary_df.replace(counter, re.sub(regex, "", job)[:-1])
#     counter += 1
    
# print(salary_df['JobTitle'].value_counts())

TRANSIT OPERATOR                                      2388
SPECIAL NURSE                                         1402
REGISTERED NURSE                                      1219
CUSTODIAN                                              796
FIREFIGHTER                                            794
POLICE OFFICER III                                     779
RECREATION LEADER                                      692
DEPUTY SHERIFF                                         685
MENTAL HEALTH REHABILITATION WORKER                    559
PUBLIC SERVICE AIDE-SPECIAL PROGRAMS                   531
POLICE OFFICER I                                       467
ATTORNEY (CIVIL/CRIMINAL)                              377
GENERAL LABORER                                        377
PORTER                                                 370
POLICE OFFICER II                                      361
PUBLIC SERVICE TRAINEE                                 328
SENIOR CLERK TYPIST                                    3

C:\Users\UNKNWN\Anaconda3\envs\37cogs108\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


TRANSIT OPERATOR                             2388
SPECIAL NURSE                                1402
REGISTERED NURSE                             1219
CUSTODIAN                                     796
FIREFIGHTER                                   794
POLICE OFFICER III                            779
RECREATION LEADER                             692
DEPUTY SHERIFF                                685
MENTAL HEALTH REHABILITATION WORKER           559
PUBLIC SERVICE AIDE-SPECIAL PROGRAMS          531
POLICE OFFICER I                              467
GENERAL LABORER                               377
ATTORNEY (CIVIL/CRIMINAL)                     377
PORTER                                        370
POLICE OFFICER II                             361
PUBLIC SERVICE TRAINEE                        328
SENIOR CLERK TYPIST                           310
SENIOR ELIGIBILITY WORKER                     308
LIBRARY PAGE                                  308
FIRE FIGHTER PARAMEDIC                        301


In [12]:
#salary_2011["JobTitle"].value_counts().index.tolist()
#print("POLICE" in "POLICE OFFICER")

['TRANSIT OPERATOR',
 'SPECIAL NURSE',
 'REGISTERED NURSE',
 'CUSTODIAN',
 'FIREFIGHTER',
 'POLICE OFFICER III',
 'RECREATION LEADER',
 'DEPUTY SHERIFF',
 'MENTAL HEALTH REHABILITATION WORKER',
 'PUBLIC SERVICE AIDE-SPECIAL PROGRAMS',
 'POLICE OFFICER I',
 'GENERAL LABORER',
 'ATTORNEY (CIVIL/CRIMINAL)',
 'PORTER',
 'POLICE OFFICER II',
 'PUBLIC SERVICE TRAINEE',
 'SENIOR CLERK TYPIST',
 'SENIOR ELIGIBILITY WORKER',
 'LIBRARY PAGE',
 'FIRE FIGHTER PARAMEDIC',
 'SERGEANT III (POLICE DEPARTMENT)',
 'GARDENER',
 'PARKING CONTROL OFFICER',
 'STATIONARY ENGINEER',
 'SENIOR CLERK',
 'ELIGIBILITY WORKER',
 'CLERK',
 'MUSEUM GUARD',
 'PROTECTIVE SERVICES WORKER',
 'NURSE PRACTITIONER',
 'ELECTRICAL TRANSIT SYSTEM MECHANIC',
 'TRANSIT SUPERVISOR',
 'LICENSED VOCATIONAL NURSE',
 'SENIOR ADMINISTRATIVE ANALYST',
 'AIRPORT POLICE SERVICES AIDE',
 'TRUCK DRIVER',
 'LIBRARIAN I',
 'PUBLIC SERVICE AIDE-ASSOCIATE TO PROFESSIONALS',
 'LIEUTENANT, FIRE DEPARTMENT',
 'MEDICAL EVALUATIONS ASSISTANT',
 'DE

We'll go ahead and now sort the thousands of jobs into a few catagories to make viewing and calculations easier

In [44]:
def groupSorter(seriesToSort):
    policeNetwork = []
    fireNetwork = []
    healthNetwork = []
    serviceNetwork = []
    lawNetwork= []
    commerceNetwork = []
    adminNetwork = []
    itNetwork = []
    otherNetwork = []
    for index, row in seriesToSort.iterrows():
        obj = row['JobTitle']
        #print(row)
        #Put these in Police network
        if "POLICE" in obj:
            policeNetwork.append(index)
        elif "SHERIFF" in obj:
            policeNetwork.append(index)
        elif "OFFICER" in obj:
            policeNetwork.append(index)
        elif "GUARD" in obj:
            if "LIFEGUARD" in obj:
                serviceNetwork.append(index)
            else:
                policeNetwork.append(index)
        elif "COMMISSIONER" in obj:
            policeNetwork.append(index)
        #put these in the fire network
        elif "FIRE" in obj:
            fireNetwork.append(index)
        #Put these in health Network
        elif "NURS" in obj:
            healthNetwork.append(index)
        elif "PHARM" in obj:
            healthNetwork.append(index)
        elif "HEALTH" in obj:
            healthNetwork.append(index)
        elif "PYSCH" in obj:
            healthNetwork.append(index)
        elif "MEDICAL" in obj:
            healthNetwork.append(index)
        #Put these in service network
        elif "DRIVE" in obj:
            serviceNetwork.append(index)
        elif "CUST" in obj:
            serviceNetwork.append(index)
        elif "REC" in obj:
            serviceNetwork.append(index)
        elif "LIBRAR" in obj:
            serviceNetwork.append(index)
        elif "SERVICE" in obj:
            serviceNetwork.append(index)
        elif "ELIGIBILTY" in obj:
            serviceNetwork.append(index)
        elif "GARDNER" in obj:
            serviceNetwork.append(index)
        elif "WORKER" in obj:
            serviceNetwork.append(index)
        elif "TECHNICIAN" in obj:
            serviceNetwork.append(index)
        elif "OPERATOR" in obj:
            serviceNetwork.append(index)
        #Put these in law
        elif "LAWYER" in obj:
            lawNetwork.append(index)
        elif "ATTORNEY" in obj:
            lawNetwork.append(index)
        elif "SAFETY" in obj:
            lawNetwork.append(index)
        elif "COURT" in obj:
            lawNetwork.append(index)
        #put these in commerce
        elif "STORE" in obj:
            commerceNetwork.append(index)
        #put these in adminstrative
        elif "ADMIN" in obj:
            adminNetwork.append(index)
        elif "SUPER" in obj:
            adminNetwork.append(index)
        elif "AIDE" in obj:
            adminNetwork.append(index)
        elif "MANAG" in obj:
            adminNetwork.append(index)
        elif "SPECIAL" in obj:
            adminNetwork.append(index)
        elif "CLERK" in obj:
            adminNetwork.append(index)
        #put these in IT
        elif "ENGINEER" in obj:
            itNetwork.append(index)
        elif "IS" in obj:
            itNetwork.append(index)
        #put these in other
        else:
            otherNetwork.append(index)

    return policeNetwork, fireNetwork, healthNetwork, serviceNetwork, lawNetwork, commerceNetwork, adminNetwork, itNetwork, otherNetwork


In [45]:
policeGrouped_2011 = []
fireGrouped_2011 = []
healthGrouped_2011 = []
servicesGrouped_2011 = []
lawGrouped_2011 = []
commerceGrouped_2011 = []
adminGrouped_2011 = []
itGrouped_2011 = []
otherGrouped_2011 = []

policeGrouped_2011, fireGrouped_2011, healthGrouped_2011, servicesGrouped_2011, lawGrouped_2011, commerceGrouped_2011, adminGrouped_2011, itGrouped_2011, otherGrouped_2011 = groupSorter(salary_2011)

policeGrouped_2012= []
fireGrouped_2012= []
healthGrouped_2012= []
servicesGrouped_2012= []
lawGrouped_2012= []
commerceGrouped_2012= []
adminGrouped_2012= []
itGrouped_2012= []
otherGrouped_2012= []

policeGrouped_2012, fireGrouped_2012, healthGrouped_2012, servicesGrouped_2012, lawGrouped_2012, commerceGrouped_2012, adminGrouped_2012, itGrouped_2012, otherGrouped_2012 = groupSorter(salary_2012)


policeGrouped_2013= []
fireGrouped_2013= []
healthGrouped_2013= []
servicesGrouped_2013= []
lawGrouped_2013= []
commerceGrouped_2013= []
adminGrouped_2013= []
itGrouped_2013= []
otherGrouped_2013= []

policeGrouped_2013, fireGrouped_2013, healthGrouped_2013, servicesGrouped_2013, lawGrouped_2013, commerceGrouped_2013, adminGrouped_2013, itGrouped_2013, otherGrouped_2013 = groupSorter(salary_2013)

policeGrouped_2014= []
fireGrouped_2014= []
healthGrouped_2014= []
servicesGrouped_2014= []
lawGrouped_2014= []
commerceGrouped_2014= []
adminGrouped_2014= []
itGrouped_2014= []
otherGrouped_2014= []

policeGrouped_2014, fireGrouped_2014, healthGrouped_2014, servicesGrouped_2014, lawGrouped_2014, commerceGrouped_2014, adminGrouped_2014, itGrouped_2014, otherGrouped_2014 = groupSorter(salary_2014)

Lets generate averages and new dataframes for the departmental data

In [60]:
#Set up averages and new Dataframes

averagePolice2011 = 0
averagePolice2012 = 0
averagePolice2013 = 0
averagePolice2014 = 0

police2011list = []
police2012list = []
police2013list = []
police2014list = []

for i in policeGrouped_2011:
    averagePolice2011 += salary_2011['TotalPayBenefits'].loc[i]
    police2011list.append(salary_2011.loc[i])
    
for i in policeGrouped_2012:
    averagePolice2012 += salary_2012['TotalPayBenefits'].loc[i]
    police2012list.append(salary_2012.loc[i])
    
for i in policeGrouped_2013:
    averagePolice2013 += salary_2013['TotalPayBenefits'].loc[i]
    police2013list.append(salary_2013.loc[i])
    
for i in policeGrouped_2014:
    averagePolice2014 += salary_2014['TotalPayBenefits'].loc[i]
    police2014list.append(salary_2014.loc[i])
    
    
averagePolice2011 = averagePolice2011/len(policeGrouped_2011)
averagePolice2012 = averagePolice2012/len(policeGrouped_2012)
averagePolice2013 = averagePolice2013/len(policeGrouped_2013)
averagePolice2014 = averagePolice2014/len(policeGrouped_2014)

police2011df = pd.DataFrame(police2011list)
police2012df = pd.DataFrame(police2012list)
police2013df = pd.DataFrame(police2013list)
police2014df = pd.DataFrame(police2014list)

In [61]:
averageFire2011 = 0
averageFire2012 = 0
averageFire2013 = 0
averageFire2014 = 0

fire2011list = []
fire2012list = []
fire2013list = []
fire2014list = []

for i in fireGrouped_2011:
    averageFire2011 += salary_2011["TotalPayBenefits"].loc[i]
    fire2011list.append(salary_2011.loc[i])
    
for i in fireGrouped_2012:
    averageFire2012 += salary_2012["TotalPayBenefits"].loc[i]
    fire2012list.append(salary_2012.loc[i])
    
for i in fireGrouped_2013:
    averageFire2013 += salary_2013["TotalPayBenefits"].loc[i]
    fire2013list.append(salary_2013.loc[i])
    
for i in fireGrouped_2014:
    averageFire2014 += salary_2014["TotalPayBenefits"].loc[i]
    fire2014list.append(salary_2014.loc[i])
    
averageFire2011 = averageFire2011/len(fireGrouped_2011)
averageFire2012 = averageFire2012/len(fireGrouped_2012)
averageFire2013 = averageFire2013/len(fireGrouped_2013)
averageFire2014 = averageFire2014/len(fireGrouped_2014)

fire2011df = pd.DataFrame(fire2011list)
fire2012df = pd.DataFrame(fire2012list)
fire2013df = pd.DataFrame(fire2013list)
fire2014df = pd.DataFrame(fire2014list)

In [62]:
averageHealth2011 = 0
averageHealth2012 = 0
averageHealth2013 = 0
averageHealth2014 = 0

health2011list = []
health2012list = []
health2013list = []
health2014list = []

for i in healthGrouped_2011:
    averageHealth2011 += salary_2011["TotalPayBenefits"].loc[i]
    health2011list.append(salary_2011.loc[i])
    
for i in healthGrouped_2012:
    averageHealth2012 += salary_2012["TotalPayBenefits"].loc[i]
    health2012list.append(salary_2012.loc[i])
    
for i in healthGrouped_2013:
    averageHealth2013 += salary_2013["TotalPayBenefits"].loc[i]
    health2013list.append(salary_2013.loc[i])
    
for i in healthGrouped_2014:
    averageHealth2014 += salary_2014["TotalPayBenefits"].loc[i]
    health2014list.append(salary_2014.loc[i])
    
averageHealth2011 = averageHealth2011/len(healthGrouped_2011)
averageHealth2012 = averageHealth2012/len(healthGrouped_2012)
averageHealth2013 = averageHealth2013/len(healthGrouped_2013)
averageHealth2014 = averageHealth2014/len(healthGrouped_2014)

health2011df = pd.DataFrame(health2011list)
health2012df = pd.DataFrame(health2012list)
health2013df = pd.DataFrame(health2013list)
health2014df = pd.DataFrame(health2014list)

In [74]:
averageService2011 = 0
averageService2012 = 0
averageService2013 = 0
averageService2014 = 0

service2011list = []
service2012list = []
service2013list = []
service2014list = []

for i in servicesGrouped_2011:
    averageService2011 += salary_2011["TotalPayBenefits"].loc[i]
    service2011list.append(salary_2011.loc[i])
    
for i in servicesGrouped_2012:
    averageService2012 += salary_2012["TotalPayBenefits"].loc[i]
    service2012list.append(salary_2012.loc[i])
    
for i in servicesGrouped_2013:
    averageService2013 += salary_2013["TotalPayBenefits"].loc[i]
    service2013list.append(salary_2013.loc[i])
    
for i in servicesGrouped_2014:
    averageService2014 += salary_2014["TotalPayBenefits"].loc[i]
    service2014list.append(salary_2014.loc[i])
    
averageService2011 = averageService2011/len(servicesGrouped_2011)
averageService2012 = averageService2012/len(servicesGrouped_2012)
averageService2013 = averageService2013/len(servicesGrouped_2013)
averageService2014 = averageService2014/len(servicesGrouped_2014)

service2011df = pd.DataFrame(service2011list)
service2012df = pd.DataFrame(service2012list)
service2013df = pd.DataFrame(service2013list)
service2014df = pd.DataFrame(service2014list)

In [65]:
averageLaw2011 = 0
averageLaw2012 = 0
averageLaw2013 = 0
averageLaw2014 = 0

law2011list = []
law2012list = []
law2013list = []
law2014list = []

for i in lawGrouped_2011:
    averageLaw2011 += salary_2011["TotalPayBenefits"].loc[i]
    law2011list.append(salary_2011.loc[i])
    
for i in lawGrouped_2012:
    averageLaw2012 += salary_2012["TotalPayBenefits"].loc[i]
    law2012list.append(salary_2012.loc[i])
    
for i in lawGrouped_2013:
    averageLaw2013 += salary_2013["TotalPayBenefits"].loc[i]
    law2013list.append(salary_2013.loc[i])
    
for i in lawGrouped_2014:
    averageLaw2014 += salary_2014["TotalPayBenefits"].loc[i]
    law2014list.append(salary_2014.loc[i])
    
averageLaw2011 = averageLaw2011/len(lawGrouped_2011)
averageLaw2012 = averageLaw2012/len(lawGrouped_2012)
averageLaw2013 = averageLaw2013/len(lawGrouped_2013)
averageLaw2014 = averageLaw2014/len(lawGrouped_2014)

law2011df = pd.DataFrame(law2011list)
law2012df = pd.DataFrame(law2012list)
law2013df = pd.DataFrame(law2013list)
law2014df = pd.DataFrame(law2014list)

In [66]:
averageCommerce2011 = 0
averageCommerce2012 = 0
averageCommerce2013 = 0
averageCommerce2014 = 0

commerce2011list = []
commerce2012list = []
commerce2013list = []
commerce2014list = []

for i in commerceGrouped_2011:
    averageCommerce2011 += salary_2011["TotalPayBenefits"].loc[i]
    commerce2011list.append(salary_2011.loc[i])
    
for i in commerceGrouped_2012:
    averageCommerce2012 += salary_2012["TotalPayBenefits"].loc[i]
    commerce2012list.append(salary_2012.loc[i])
    
for i in commerceGrouped_2013:
    averageCommerce2013 += salary_2013["TotalPayBenefits"].loc[i]
    commerce2013list.append(salary_2013.loc[i])
    
for i in commerceGrouped_2014:
    averageCommerce2014 += salary_2014["TotalPayBenefits"].loc[i]
    commerce2014list.append(salary_2014.loc[i])
    
averageCommerce2011 = averageCommerce2011/len(commerceGrouped_2011)
averageCommerce2012 = averageCommerce2012/len(commerceGrouped_2012)
averageCommerce2013 = averageCommerce2013/len(commerceGrouped_2013)
averageCommerce2014 = averageCommerce2014/len(commerceGrouped_2014)

commerce2011df = pd.DataFrame(commerce2011list)
commerce2012df = pd.DataFrame(commerce2012list)
commerce2013df = pd.DataFrame(commerce2013list)
commerce2014df = pd.DataFrame(commerce2014list)

In [67]:
averageAdmin2011 = 0
averageAdmin2012 = 0
averageAdmin2013 = 0
averageAdmin2014 = 0

admin2011list = []
admin2012list = []
admin2013list = []
admin2014list = []

for i in adminGrouped_2011:
    averageAdmin2011 += salary_2011["TotalPayBenefits"].loc[i]
    admin2011list.append(salary_2011.loc[i])
    
for i in adminGrouped_2012:
    averageAdmin2012 += salary_2012["TotalPayBenefits"].loc[i]
    admin2012list.append(salary_2012.loc[i])
    
for i in adminGrouped_2013:
    averageAdmin2013 += salary_2013["TotalPayBenefits"].loc[i]
    admin2013list.append(salary_2013.loc[i])
    
for i in adminGrouped_2014:
    averageAdmin2014 += salary_2014["TotalPayBenefits"].loc[i]
    admin2014list.append(salary_2014.loc[i])
    
averageAdmin2011 = averageAdmin2011/len(adminGrouped_2011)
averageAdmin2012 = averageAdmin2012/len(adminGrouped_2012)
averageAdmin2013 = averageAdmin2013/len(adminGrouped_2013)
averageAdmin2014 = averageAdmin2014/len(adminGrouped_2014)

admin2011df = pd.DataFrame(admin2011list)
admin2012df = pd.DataFrame(admin2012list)
admin2013df = pd.DataFrame(admin2013list)
admin2014df = pd.DataFrame(admin2014list)

In [71]:
averageIt2011 = 0
averageIt2012 = 0
averageIt2013 = 0
averageIt2014 = 0

It2011list = []
It2012list = []
It2013list = []
It2014list = []

for i in itGrouped_2011:
    averageIt2011 += salary_2011["TotalPayBenefits"].loc[i]
    It2011list.append(salary_2011.loc[i])
    
for i in itGrouped_2012:
    averageIt2012 += salary_2012["TotalPayBenefits"].loc[i]
    It2012list.append(salary_2012.loc[i])
    
for i in itGrouped_2013:
    averageIt2013 += salary_2013["TotalPayBenefits"].loc[i]
    It2013list.append(salary_2013.loc[i])
    
for i in itGrouped_2014:
    averageIt2014 += salary_2014["TotalPayBenefits"].loc[i]
    It2014list.append(salary_2014.loc[i])
    
averageIt2011 = averageIt2011/len(itGrouped_2011)
averageIt2012 = averageIt2012/len(itGrouped_2012)
averageIt2013 = averageIt2013/len(itGrouped_2013)
averageIt2014 = averageIt2014/len(itGrouped_2014)

It2011df = pd.DataFrame(It2011list)
It2012df = pd.DataFrame(It2012list)
It2013df = pd.DataFrame(It2013list)
It2014df = pd.DataFrame(It2014list)

In [72]:
averageOther2011 = 0
averageOther2012 = 0
averageOther2013 = 0
averageOther2014 = 0

other2011list = []
other2012list = []
other2013list = []
other2014list = []

for i in otherGrouped_2011:
    averageOther2011 += salary_2011["TotalPayBenefits"].loc[i]
    other2011list.append(salary_2011.loc[i])
    
for i in otherGrouped_2012:
    averageOther2012 += salary_2012["TotalPayBenefits"].loc[i]
    other2012list.append(salary_2012.loc[i])
    
for i in otherGrouped_2013:
    averageOther2013 += salary_2013["TotalPayBenefits"].loc[i]
    other2013list.append(salary_2013.loc[i])
    
for i in otherGrouped_2014:
    averageOther2014 += salary_2014["TotalPayBenefits"].loc[i]
    other2014list.append(salary_2014.loc[i])
    
averageOther2011 = averageOther2011/len(otherGrouped_2011)
averageOther2012 = averageOther2012/len(otherGrouped_2012)
averageOther2013 = averageOther2013/len(otherGrouped_2013)
averageOther2014 = averageOther2014/len(otherGrouped_2014)

other2011df = pd.DataFrame(other2011list)
other2012df = pd.DataFrame(other2012list)
other2013df = pd.DataFrame(other2013list)
other2014df = pd.DataFrame(other2014list)